# Predictiong Ground-state with Novel Quantum Descriptor of Molecules using QML

## Can we get grond state enery with only Cartesian coordinates of atoms in molecule?
<img src="carcorr.png" alt="drawing" width="400">

Predicting ground-state enery of molecules are hard yet important problem in quantum chemistry. Even if the physical structure of a molecule is known, directly calculating ground-state from quantum mechanics is challenging. Therefore, many approximate classical algorithms have been developed with neural network, such as ASCF, SOAP, etc. In this QHack project, we demonstrate quantum-classical hybrid machine learning algorithm predicting ground-state only with Cartesian coordinates and atom number of atoms in a molecule.



### (High-dimensional) Neural Network Potential
[The paper](https://pubs.acs.org/action/showCitFormats?doi=10.1021/acs.chemrev.0c00868&ref=pdf) suggests Neural Network Potential (NNP) to get ground-state. For atom $\alpha$ in molecule, there are $N_\alpha$ adjacent atoms within some cut-off radius $R_c$. Then, we transform distance from these adjacent atoms by some symmetry functions, called descriptors; $R_\alpha \mapsto G_\alpha$. For example, typical radial symmetry preserving descriptor would be,
$$
    G^{rad}_{i, \mu} = \sum_{j\neq i}^{N_{atom}\in R_c} e^{-\eta(R_{ij}-R_s)^2} f_c(R_{ij})
$$
where $f_c$ is cut-off function. Finally, using classical neural network, we predict atomic energy per atom per element and sum them all to get (short-interaction) energy. We have to train these neural network with molecules of well-known ground-state energy. In NNP, design of the discriptors are extremely important.

![2ndNNP](https://pubs.acs.org/na101/home/literatum/publisher/achs/journals/content/chreay/2021/chreay.2021.121.issue-16/acs.chemrev.0c00868/20210818/images/medium/cr0c00868_0005.gif)

## Let's go quantum! (QNNP)
Quantum Neural Network Potential (QNNP) utilized quantum descriptor instead of classical descriptor,
$$
    |G_\alpha^i\rangle = \sum_{j\neq i}^{N_{atom}\in R_c}e^{-\eta(R_{ij}-R_s)^2}f_c(R_{ij})|\psi_{\alpha,j}\rangle.
$$
Here, $|\psi_{\alpha,j}\rangle$ is some kind of projection mapping onto Bloch sphere.

<img src="desc.png" alt="drawing" width="300">

Since descriptors are quantum, we utilze parameterized quantum circuit instead of classical neural network.

<img src="qcirc.png" alt="drawing" width="1000">

## Demo

We trained our QNNP model with [QM9 dataset](https://paperswithcode.com/dataset/qm9)

In [1]:
from QMLatomLoader import *
import pennylane as qml
from pennylane import numpy as np
from tqdm.notebook import tqdm
import json
from torch.utils.tensorboard import SummaryWriter

Let's consider one-qubit case for simplicity.

In [2]:
epochs = 2000
batchs = 32

n_qubit = 1
radius = 10

para = np.array(
    [[radius, 0],
     ]
)
para.requires_grad = False
print(para)

[[10  0]]


We consider default pennylane qubit simulator

In [3]:
dev = qml.device("default.qubit", wires=n_qubit)


@qml.qnode(dev)
def atomicoutput(descript, H):
    """The energy expectation value of a Hamiltonian"""
    for i in range(n_qubit):
        qml.Rot(-descript[i, 1], descript[i, 0], descript[i, 1], wires=i)

    return qml.expval(H)


def hamiltonian(parameters, descriptor_size):
    coeff = np.zeros((n_qubit, 4)) + descriptor_size[:, np.newaxis]
    coeff = coeff.flatten()
    coeff = coeff * parameters
    obs = []
    for i in range(n_qubit):
        obs.append(qml.Identity(i))
        obs.append(qml.PauliX(i))
        obs.append(qml.PauliY(i))
        obs.append(qml.PauliZ(i))
    return qml.Hamiltonian(coeff, obs)

We use optimizers provided in `pennylane` package

In [4]:
writer = SummaryWriter()
init_params = np.random.random(n_qubit * 4, requires_grad=True)
opt = qml.AdagradOptimizer(stepsize=2)
params = init_params
params.requires_grad = True
print(params)

[0.56228473 0.9649423  0.78495708 0.78705951]


In [ ]:
losslist = []
if batchs == 1:
    print('loading atom data')
    loadatom = AtomLoader1(
        sampler='random',
        numb=epochs,
        classic=True,
        classic_parameter=para,
        weigthed=True,
        cutoff_radius=radius,
        set_axis=True
    )
    print('atom data loaded!')
    for i in tqdm(range(epochs)):
        ground_energy = loadatom[i]['ground_energy']
        descriptor = loadatom[i]['descriptor']
        descript_sizes = loadatom[i]['descriptor_size']
        n_atom = loadatom[i]['atomic_number']
        descriptor.requires_grad = False
        descript_sizes.requires_grad = False


        def losses(param):
            outputs = 0
            for n in range(n_atom):
                outputs += atomicoutput(descriptor[n], hamiltonian(param, descript_sizes[n]))
            return np.sqrt(((ground_energy - outputs) / n_atom) ** 2)


        params, loss = opt.step_and_cost(losses, params)
        writer.add_scalar(f'Batch={batchs}/Loss', loss, i)
        losslist.append(loss.item())
else:
    print('loading atom data')
    loadatom = AtomLoader2(
        sampler='random',
        epochs=epochs,
        batchs=batchs,
        classic=True,
        classic_parameter=para,
        weigthed=True,
        cutoff_radius=radius,
        set_axis=True
    )
    print('atom data loaded!')
    for i in tqdm(range(epochs)):
        def losses(param):
            loss = 0
            x = loadatom[i]
            for j in range(batchs):
                ground_energy = x[j]['ground_energy']
                descriptor = x[j]['descriptor']
                descript_sizes = x[j]['descriptor_size']
                n_atom = x[j]['atomic_number']
                descriptor.requires_grad = False
                descript_sizes.requires_grad = False
                out = 0
                for n in range(n_atom):
                    out += atomicoutput(descriptor[n], hamiltonian(param, descript_sizes[n]))
                loss += np.sqrt(((ground_energy - out) / n_atom) ** 2)
            return loss / batchs


        params, loss = opt.step_and_cost(losses, params)
        writer.add_scalar(f'Batch={batchs}/Loss', loss, i)
        losslist.append(loss.item())

loading atom data
atom data loaded!


  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
print(para)
print(params)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
data = {'step': np.arange(len(losslist)), 'loss':np.array(losslist)}
sns.lineplot(data=data, x='step', y='loss')

## Ref
1. [Behler, Jörg. "Four generations of high-dimensional neural network potentials." Chemical Reviews 121.16 (2021): 10037-10072.](https://doi.org/10.1021/acs.chemrev.0c00868)